# 🤝 Species Interactions
## How Organisms Interact in Communities

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-3-community/notebooks/01_species_interactions.ipynb)

---

> *"In nature, nothing exists alone. Every species is connected through an intricate web of interactions."*

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Identify the **6 major types** of species interactions
2. Understand **symbiosis** and its forms
3. Analyze **competition** using the competitive exclusion principle
4. Explore **predator-prey dynamics** with Lotka-Volterra models
5. Recognize **mutualism** and its ecological importance
6. Distinguish between **+/+, +/0, +/-, -/-, and 0/- interactions**
7. Apply interaction concepts to real ecological examples

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib scipy networkx -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
from scipy.integrate import odeint

print("✅ Ready to explore species interactions!")
print("🤝 Let's discover how organisms interact!")

---

## 📊 Part 1: The Six Types of Species Interactions

### Overview:

Every species interaction can be classified by its effect on each species:
- **+** = Positive effect (benefits)
- **0** = Neutral effect (no impact)
- **-** = Negative effect (harms)

### The Interaction Matrix:

| Interaction | Species A | Species B | Symbol | Example |
|-------------|-----------|-----------|--------|----------|
| **Mutualism** | + | + | +/+ | Bees & flowers |
| **Competition** | - | - | -/- | Lions & hyenas |
| **Predation** | + | - | +/- | Wolf & deer |
| **Herbivory** | + | - | +/- | Deer & grass |
| **Parasitism** | + | - | +/- | Tick & dog |
| **Commensalism** | + | 0 | +/0 | Barnacle & whale |
| **Amensalism** | 0 | - | 0/- | Large tree & seedling |

### Symbiosis:
**Definition**: Close, long-term interaction between two species

Includes:
- Mutualism (+/+)
- Commensalism (+/0)
- Parasitism (+/-)

*Note: Predation is NOT symbiosis (interaction is brief)*

In [ ]:
# Visualize interaction types
interactions = {
    'Mutualism': {'A': '+', 'B': '+', 'color': 'green', 'example': 'Bee ↔ Flower'},
    'Competition': {'A': '-', 'B': '-', 'color': 'red', 'example': 'Lion ↔ Hyena'},
    'Predation': {'A': '+', 'B': '-', 'color': 'orange', 'example': 'Wolf → Deer'},
    'Herbivory': {'A': '+', 'B': '-', 'color': 'yellow', 'example': 'Deer → Grass'},
    'Parasitism': {'A': '+', 'B': '-', 'color': 'purple', 'example': 'Tick → Dog'},
    'Commensalism': {'A': '+', 'B': '0', 'color': 'lightblue', 'example': 'Barnacle → Whale'},
    'Amensalism': {'A': '0', 'B': '-', 'color': 'gray', 'example': 'Tree → Seedling'}
}

# Create interaction matrix visualization
interaction_names = list(interactions.keys())
species_a = [interactions[i]['A'] for i in interaction_names]
species_b = [interactions[i]['B'] for i in interaction_names]
colors = [interactions[i]['color'] for i in interaction_names]
examples = [interactions[i]['example'] for i in interaction_names]

# Create table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>Interaction</b>', '<b>Species A</b>', '<b>Species B</b>', '<b>Type</b>', '<b>Example</b>'],
        fill_color='lightblue',
        align='left',
        font=dict(size=14, color='black')
    ),
    cells=dict(
        values=[
            interaction_names,
            species_a,
            species_b,
            [f"{a}/{b}" for a, b in zip(species_a, species_b)],
            examples
        ],
        fill_color=[colors] * 5,
        align='left',
        font=dict(size=13),
        height=35
    )
)])

fig.update_layout(
    title="🤝 The Seven Types of Species Interactions",
    height=400
)

fig.show()

print("\n💡 Key Points:")
print("   • Mutualism (+/+): Both benefit")
print("   • Competition (-/-): Both harmed")
print("   • Predation/Parasitism (+/-): One benefits, one harmed")
print("   • Commensalism (+/0): One benefits, other unaffected")
print("   • Amensalism (0/-): One unaffected, other harmed")

---

## 🥊 Part 2: Competition (-/-)

### Definition:
**Competition**: When two or more organisms use the same limited resource

### Two Types:

#### 1. **Intraspecific Competition** (within species)
- Members of SAME species compete
- Examples: Trees competing for light, lions competing for territory
- Usually STRONGER than interspecific

#### 2. **Interspecific Competition** (between species)
- Members of DIFFERENT species compete
- Examples: Lions vs hyenas for prey
- Can lead to competitive exclusion

### Competitive Exclusion Principle (Gause's Law):

**"Two species competing for the same limited resource cannot coexist indefinitely."**

**Outcome**: One species will outcompete and eliminate the other

### Famous Experiment: Gause's Paramecium (1934)

- **Species**: *Paramecium aurelia* and *P. caudatum*
- **Setup**: Grown separately → both thrive
- **Together**: *P. aurelia* always wins
- **Reason**: *P. aurelia* is slightly more efficient
- **Result**: *P. caudatum* goes extinct

In [ ]:
# Simulate competitive exclusion (Lotka-Volterra competition)
def competition_model(state, t, r1, r2, K1, K2, alpha, beta):
    """
    Lotka-Volterra competition model
    r = growth rate
    K = carrying capacity
    alpha = effect of species 2 on species 1
    beta = effect of species 1 on species 2
    """
    N1, N2 = state
    
    dN1_dt = r1 * N1 * (K1 - N1 - alpha * N2) / K1
    dN2_dt = r2 * N2 * (K2 - N2 - beta * N1) / K2
    
    return [dN1_dt, dN2_dt]

# Scenario 1: Competitive exclusion (Species 1 wins)
t = np.linspace(0, 100, 1000)
initial_state = [10, 10]

# Parameters for exclusion
r1, r2 = 1.0, 0.8  # Species 1 grows faster
K1, K2 = 1000, 1000
alpha, beta = 1.2, 0.8  # Species 1 suppresses 2 more than vice versa

solution = odeint(competition_model, initial_state, t, args=(r1, r2, K1, K2, alpha, beta))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t, y=solution[:, 0],
    mode='lines',
    line=dict(color='blue', width=3),
    name='Species 1 (Stronger competitor)'
))

fig.add_trace(go.Scatter(
    x=t, y=solution[:, 1],
    mode='lines',
    line=dict(color='red', width=3),
    name='Species 2 (Weaker competitor)'
))

fig.update_layout(
    title="🥊 Competitive Exclusion: Gause's Principle in Action<br><sub>Species 2 driven to extinction</sub>",
    xaxis_title="Time",
    yaxis_title="Population Size",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n📊 Competitive Exclusion Results:")
print(f"   Species 1 (Blue): Survives at {solution[-1, 0]:.0f} individuals")
print(f"   Species 2 (Red): Goes extinct (population → {solution[-1, 1]:.2f})")
print("\n💡 This demonstrates Gause's Principle:")
print("   Two species competing for the same resource cannot coexist.")
print("   The superior competitor eliminates the inferior one.")

In [ ]:
# Scenario 2: Coexistence (resource partitioning)
# When species use resources differently, they can coexist

# Parameters for coexistence
r1, r2 = 1.0, 1.0
K1, K2 = 1000, 1000
alpha, beta = 0.5, 0.5  # Weak interspecific competition

solution_coexist = odeint(competition_model, initial_state, t, args=(r1, r2, K1, K2, alpha, beta))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t, y=solution_coexist[:, 0],
    mode='lines',
    line=dict(color='green', width=3),
    name='Species 1'
))

fig.add_trace(go.Scatter(
    x=t, y=solution_coexist[:, 1],
    mode='lines',
    line=dict(color='purple', width=3),
    name='Species 2'
))

fig.update_layout(
    title="🤝 Coexistence Through Resource Partitioning<br><sub>Both species survive by using different resources</sub>",
    xaxis_title="Time",
    yaxis_title="Population Size",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n📊 Coexistence Results:")
print(f"   Species 1: Stable at {solution_coexist[-1, 0]:.0f} individuals")
print(f"   Species 2: Stable at {solution_coexist[-1, 1]:.0f} individuals")
print("\n💡 How They Coexist:")
print("   • Resource partitioning (use different resources)")
print("   • Temporal separation (active at different times)")
print("   • Spatial separation (use different habitats)")
print("\n🌳 Real Example: Warblers in trees")
print("   5 warbler species coexist by feeding at different heights!")

---

## 🐺 Part 3: Predation (+/-)

### Definition:
**Predation**: One organism (predator) kills and eats another (prey)

### Includes:
- **True predation**: Lion eating zebra
- **Herbivory**: Deer eating grass (plant "predation")
- **Parasitoidism**: Wasp larvae eating caterpillar from inside

### Predator-Prey Cycles:

Classic pattern:
```
1. Prey abundant → Predators increase
2. Predators abundant → Prey decrease
3. Prey scarce → Predators starve and decrease
4. Predators scarce → Prey recover
5. Repeat!
```

### Famous Example: Lynx and Snowshoe Hare

Hudson Bay Company fur records (1845-1935) show:
- Populations cycle with ~10-year period
- Lynx peaks lag hare peaks by 1-2 years
- Classic predator-prey oscillation

### Lotka-Volterra Predator-Prey Model:

**Prey equation**: dN/dt = rN - aNP  
**Predator equation**: dP/dt = baNP - mP

Where:
- N = prey population
- P = predator population
- r = prey growth rate
- a = predation rate
- b = conversion efficiency
- m = predator mortality

In [ ]:
# Lotka-Volterra Predator-Prey Model
def predator_prey(state, t, r, a, b, m):
    """
    Lotka-Volterra predator-prey model
    """
    N, P = state  # Prey (N), Predator (P)
    
    dN_dt = r * N - a * N * P  # Prey growth - predation
    dP_dt = b * a * N * P - m * P  # Predator growth from eating - mortality
    
    return [dN_dt, dP_dt]

# Parameters
r = 1.0   # Prey growth rate
a = 0.01  # Predation rate
b = 0.5   # Conversion efficiency
m = 0.2   # Predator mortality

# Initial conditions
N0, P0 = 40, 9  # Initial prey and predator
initial = [N0, P0]

# Time
t = np.linspace(0, 200, 2000)

# Solve
solution_pp = odeint(predator_prey, initial, t, args=(r, a, b, m))

# Plot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Population Cycles Over Time', 'Phase Space (Prey vs Predator)')
)

# Time series
fig.add_trace(
    go.Scatter(x=t, y=solution_pp[:, 0], mode='lines',
               line=dict(color='green', width=2), name='Prey (Hare)'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=t, y=solution_pp[:, 1], mode='lines',
               line=dict(color='red', width=2), name='Predator (Lynx)'),
    row=1, col=1
)

# Phase space
fig.add_trace(
    go.Scatter(x=solution_pp[:, 0], y=solution_pp[:, 1],
               mode='lines', line=dict(color='purple', width=2),
               name='Cycle', showlegend=False),
    row=1, col=2
)

# Mark starting point
fig.add_trace(
    go.Scatter(x=[N0], y=[P0], mode='markers',
               marker=dict(size=15, color='blue'),
               name='Start', showlegend=False),
    row=1, col=2
)

fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_xaxes(title_text="Prey Population", row=1, col=2)
fig.update_yaxes(title_text="Population", row=1, col=1)
fig.update_yaxes(title_text="Predator Population", row=1, col=2)

fig.update_layout(
    title="🐺 Predator-Prey Dynamics: Lynx and Snowshoe Hare<br><sub>Classic cycling pattern</sub>",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n📊 Predator-Prey Cycle Observations:")
print("   • Both populations oscillate (cycle)")
print("   • Predator peaks LAG behind prey peaks")
print("   • Populations never reach equilibrium")
print("   • Classic pattern seen in many systems")
print("\n🐰🐺 Real Data (Hudson Bay, 1900s):")
print("   • Snowshoe hare: 3-year peaks")
print("   • Lynx: Peaks 1-2 years after hares")
print("   • ~10-year cycle period")

---

## 🐜 Part 4: Parasitism (+/-)

### Definition:
**Parasitism**: One organism (parasite) lives on/in and harms another (host) without immediately killing it

### Key Differences from Predation:

| Feature | Predation | Parasitism |
|---------|-----------|------------|
| **Size** | Predator usually larger | Parasite usually smaller |
| **Number** | Eats many prey | One host per parasite (usually) |
| **Speed** | Kills quickly | Harms slowly |
| **Relationship** | Brief interaction | Long-term association |

### Types of Parasites:

#### 1. **Ectoparasites** (external)
- Live ON the host
- Examples: Ticks, fleas, lice, leeches
- Easy to remove

#### 2. **Endoparasites** (internal)
- Live INSIDE the host
- Examples: Tapeworms, malaria, heartworms
- Hard to remove

### Impact:
- Reduce host fitness
- Weaken immune system
- Alter behavior
- Can regulate host populations

### Famous Examples:
- **Toxoplasma gondii**: Makes mice fearless of cats!
- **Cordyceps fungus**: "Zombie ant" fungus
- **Mistletoe**: Plant parasite on trees

---

## 🌸 Part 5: Mutualism (+/+)

### Definition:
**Mutualism**: Both species benefit from the interaction

### Types:

#### 1. **Obligate Mutualism**
- Both species NEED each other to survive
- Cannot live apart
- Examples:
  - Termites & gut bacteria (digest wood)
  - Lichens (fungus + algae)
  - Mycorrhizae (plant roots + fungi)

#### 2. **Facultative Mutualism**
- Both benefit but CAN live apart
- Examples:
  - Bees & flowers
  - Oxpeckers & large mammals
  - Clownfish & anemones

### Famous Mutualisms:

#### 🐝 **Bees and Flowers**
- Bee gets: Nectar (food)
- Flower gets: Pollination (reproduction)
- Result: Both thrive!

#### 🌳 **Mycorrhizae**
- Plant gets: Water + minerals (from fungus)
- Fungus gets: Sugars (from plant photosynthesis)
- Impact: 90% of plants have mycorrhizal partners!

#### 🐜 **Ants and Acacia Trees**
- Ant gets: Food (nectar) + shelter (hollow thorns)
- Tree gets: Protection from herbivores
- Amazing: Ants attack anything touching the tree!

#### 🐠 **Clownfish and Anemones**
- Clownfish gets: Protection from predators
- Anemone gets: Food scraps + defense from butterfly fish
- Special: Clownfish immune to anemone's sting!

In [ ]:
# Mutualism model: Both populations benefit from each other
def mutualism_model(state, t, r1, r2, K1, K2, alpha, beta):
    """
    Mutualism model where species help each other
    alpha, beta > 0 means positive effects
    """
    N1, N2 = state
    
    # Both species benefit from each other's presence
    dN1_dt = r1 * N1 * (K1 + alpha * N2 - N1) / K1
    dN2_dt = r2 * N2 * (K2 + beta * N1 - N2) / K2
    
    return [dN1_dt, dN2_dt]

# Parameters
r1, r2 = 0.5, 0.5
K1, K2 = 100, 100
alpha, beta = 50, 50  # Strong positive effects

initial_mutual = [10, 10]
t_mutual = np.linspace(0, 50, 500)

solution_mutual = odeint(mutualism_model, initial_mutual, t_mutual, args=(r1, r2, K1, K2, alpha, beta))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=t_mutual, y=solution_mutual[:, 0],
    mode='lines',
    line=dict(color='blue', width=3),
    name='Species 1 (e.g., Bee)'
))

fig.add_trace(go.Scatter(
    x=t_mutual, y=solution_mutual[:, 1],
    mode='lines',
    line=dict(color='pink', width=3),
    name='Species 2 (e.g., Flower)'
))

fig.update_layout(
    title="🌸 Mutualism: Both Species Benefit<br><sub>Both populations increase together</sub>",
    xaxis_title="Time",
    yaxis_title="Population Size",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n🌸 Mutualism Results:")
print(f"   Species 1: Grows to {solution_mutual[-1, 0]:.0f} (higher than alone!)")
print(f"   Species 2: Grows to {solution_mutual[-1, 1]:.0f} (higher than alone!)")
print("\n💡 Key Point:")
print("   • Both populations reach HIGHER levels together")
print("   • Each species helps the other thrive")
print("   • Win-win relationship!")
print("\n🌍 Ecological Importance:")
print("   • ~90% of plants depend on mycorrhizal mutualisms")
print("   • ~80% of flowering plants need animal pollinators")
print("   • Mutualisms are EVERYWHERE in nature!")

---

## 🦀 Part 6: Commensalism (+/0) and Amensalism (0/-)

### Commensalism (+/0):
One species benefits, the other is unaffected

**Examples**:
- **Barnacles on whales**: Barnacle gets transport and feeding opportunities, whale unaffected
- **Epiphytes on trees**: Orchids get support and light, tree unaffected
- **Remoras and sharks**: Remora gets food scraps, shark unaffected
- **Birds nesting in trees**: Bird gets shelter, tree unaffected

### Amensalism (0/-):
One species is harmed, the other is unaffected

**Examples**:
- **Large tree shading seedlings**: Tree unaffected, seedlings can't grow
- **Elephant trampling grass**: Elephant unaffected, grass damaged
- **Penicillium fungus**: Produces penicillin (kills bacteria), fungus unaffected
- **Black walnut trees**: Release juglone (toxic), kills nearby plants

### Why These Are Rare:

**True neutrality is uncommon!**
- Usually there's SOME effect, even if small
- Hard to prove zero effect
- May discover effects with more study

---

## 🌐 Part 7: Interaction Networks

### Real ecosystems have MULTIPLE interactions simultaneously!

In [ ]:
# Create an interaction network
G = nx.DiGraph()

# Define species and their interactions
interactions_network = [
    ('Flowers', 'Bees', 'mutualism'),
    ('Bees', 'Flowers', 'mutualism'),
    ('Deer', 'Grass', 'herbivory'),
    ('Wolf', 'Deer', 'predation'),
    ('Tick', 'Deer', 'parasitism'),
    ('Trees', 'Seedlings', 'amensalism'),
    ('Barnacles', 'Whale', 'commensalism'),
    ('Lion', 'Hyena', 'competition'),
    ('Hyena', 'Lion', 'competition')
]

# Add edges
for source, target, interaction_type in interactions_network:
    G.add_edge(source, target, type=interaction_type)

# Position nodes
pos = nx.spring_layout(G, seed=42, k=2)

# Create figure
fig = go.Figure()

# Add edges with colors based on interaction type
edge_colors = {
    'mutualism': 'green',
    'predation': 'red',
    'herbivory': 'orange',
    'parasitism': 'purple',
    'competition': 'darkred',
    'commensalism': 'blue',
    'amensalism': 'gray'
}

for source, target, data in G.edges(data=True):
    x0, y0 = pos[source]
    x1, y1 = pos[target]
    
    fig.add_trace(go.Scatter(
        x=[x0, x1],
        y=[y0, y1],
        mode='lines',
        line=dict(width=2, color=edge_colors.get(data['type'], 'black')),
        hoverinfo='none',
        showlegend=False
    ))

# Add nodes
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = list(G.nodes())

fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    marker=dict(size=30, color='lightblue', line=dict(width=2, color='black')),
    text=node_text,
    textposition='top center',
    textfont=dict(size=10),
    hoverinfo='text',
    showlegend=False
))

fig.update_layout(
    title="🌐 Ecological Interaction Network<br><sub>Multiple species, multiple interactions</sub>",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    height=600,
    template='plotly_white'
)

fig.show()

print("\n🌐 Interaction Network Legend:")
print("   🟢 Green = Mutualism (+/+)")
print("   🔴 Red = Predation (+/-)")
print("   🟠 Orange = Herbivory (+/-)")
print("   🟣 Purple = Parasitism (+/-)")
print("   🔴 Dark Red = Competition (-/-)")
print("   🔵 Blue = Commensalism (+/0)")
print("   ⚫ Gray = Amensalism (0/-)")
print("\n💡 Real ecosystems are complex networks!")
print("   • Every species interacts with multiple others")
print("   • Changes ripple through the network")
print("   • Understanding interactions is key to conservation")

---

## 🎓 Summary

### Key Takeaways:

✅ **Six interaction types**: Mutualism, Competition, Predation, Parasitism, Commensalism, Amensalism  
✅ **Competition**: Can lead to exclusion or coexistence (resource partitioning)  
✅ **Gause's Principle**: Two species cannot occupy the same niche  
✅ **Predator-prey**: Show cyclic dynamics (Lotka-Volterra)  
✅ **Parasitism**: Long-term harm without immediate death  
✅ **Mutualism**: Both benefit, extremely common in nature  
✅ **Networks**: Real ecosystems have complex interaction webs  

### Interaction Summary Table:

| Type | Effect | Strength | Example |
|------|--------|----------|----------|
| Mutualism | +/+ | Strong | Bees ↔ Flowers |
| Competition | -/- | Strong | Lions ↔ Hyenas |
| Predation | +/- | Strong | Wolf → Deer |
| Parasitism | +/- | Moderate | Tick → Dog |
| Commensalism | +/0 | Weak | Barnacle → Whale |
| Amensalism | 0/- | Weak | Tree → Seedling |

### Real-World Implications:

#### 🌾 **Agriculture**:
- Use mutualisms (nitrogen-fixing bacteria)
- Biological pest control (predation)
- Crop rotation (reduce competition)

#### 🌳 **Conservation**:
- Protect mutualisms (pollinators)
- Manage predator-prey balance
- Understand interaction networks

#### 🏥 **Medicine**:
- Combat parasites
- Use competitive exclusion (probiotics)
- Harness mutualisms (gut bacteria)

### Next Notebook:

**02_community_structure_diversity.ipynb** - Measuring and understanding community complexity!

---

<div align="center">

**Made with 💚 by The Pattern Hunter Team**

[🏠 Unit 3 Home](../../) | 
[📓 Next: Community Structure](02_community_structure_diversity.ipynb)

</div>